In [25]:
import smbclient
import json
import cv2
from pathlib import Path
import numpy as np

def mount_smb(username, password, dataset_folder, smb_json_path):
    # Optional - specify the default credentials to use on the global config object
    smbclient.ClientConfig(username=username, password=password)

    # Optional - register the credentials with a server (overrides ClientConfig for that server)
    smbclient.register_session("192.168.152.34", username=username, password=password)

    with smbclient.open_file(smb_json_path, 'r') as f:
        json_data = json.load(f)

    base_path = r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/'

    def write_to_YOLO(img, val=False, test=False):
        img_url = img['url']
        img_name = Path(img['name']).name

        if len(img['labels']) >= 1:
            bounding_box = img['labels'][0]['box2d']
            (startX, startY, endX, endY) = [bounding_box['x1'], bounding_box['y1'], bounding_box['x2'],
                                            bounding_box['y2']]
            file_path = base_path + img_url[img_url.find('05_08_20'):]
            with smbclient.open_file(file_path, 'rb') as f:
                arr = np.asarray(bytearray(f.read()), dtype=np.uint8)
                image = cv2.imdecode(arr, -1)
                (h, w) = image.shape[:2]

                image = cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)

            x_center = ((endX + startX) / 2) / w
            y_center = (endY + startY) / 2 / h
            width = (endX - startX) / w
            height = (endY - startY) / h
            klass = 0

            # save the labels
            if val:
                with open(dataset_folder + '/labels/val/' + img_name.strip('.jpg') + '.txt', 'w+') as f:
                    f.write(' '.join([str(i) for i in [klass, x_center, y_center, width, height]]))

                # save the images
                cv2.imwrite(dataset_folder + '/images/val/' + img_name, image)
                
            elif test:
                with open(dataset_folder + '/labels/test/' + img_name.strip('.jpg') + '.txt', 'w+') as f:
                    f.write(' '.join([str(i) for i in [klass, x_center, y_center, width, height]]))

                # save the images
                cv2.imwrite(dataset_folder + '/images/test/' + img_name, image)
                
            else:
                with open(dataset_folder + '/labels/train/' + img_name.strip('.jpg') + '.txt', 'w+') as f:
                    f.write(' '.join([str(i) for i in [klass, x_center, y_center, width, height]]))

                # save the images
                cv2.imwrite(dataset_folder + '/images/train/' + img_name, image)

    print('saving training images...')
    for img in json_data[1000:2600]:
        write_to_YOLO(img)

    print('saving validation images...')
    for img in json_data[2600:3050]:
        write_to_YOLO(img, val=True)

#     print('saving testing images...')
#     for img in json_data[3050:3450]:
#         write_to_YOLO(img, test=True)


In [9]:
!pip install smbprotocol

In [23]:
username = input('username:')
password = input('password:')
mount_smb(username, password, 'yolov3/video_color_images', r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_08_20/DJI_Matice_100/2020-05-08-12-41-04-DM-Color_export.json')

saving training images...
saving validation images...


In [21]:
!cd yolov3
!ls
!pwd
!python train.py --img 256 --batch 32 --epochs 20 --weights yolov3.pt --data video_color_images.yaml  --nosave

logs  Untitled.ipynb  yolov3  yolov3setup.sh
/home/jovyan/workspace
python: can't open file 'train.py': [Errno 2] No such file or directory


In [13]:
import os
import glob
def clear_dir(directory):
    files = glob.glob(directory)
    for f in files:
        os.remove(f)

clear_dir('yolov3/labels_06_03_20_NIR/images/train/*')
clear_dir('yolov3/labels_06_03_20_NIR/images/val/*')
clear_dir('yolov3/labels_06_03_20_NIR/labels/train/*')
clear_dir('yolov3/labels_06_03_20_NIR/labels/val/*')

clear_dir('yolov3/labels_05_13_20_NIR/images/train/*')
clear_dir('yolov3/labels_05_13_20_NIR/images/val/*')
clear_dir('yolov3/labels_05_13_20_NIR/labels/train/*')
clear_dir('yolov3/labels_05_13_20_NIR/labels/val/*')

In [6]:
clear_dir('yolov3/video_color_images/images/*.jpg')

In [12]:
clear_dir('yolov3/video_color_images/images/train/*')
clear_dir('yolov3/video_color_images/images/val/*')
clear_dir('yolov3/video_color_images/labels/train/*')
clear_dir('yolov3/video_color_images/labels/val/*')

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
